<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/ALS_QNN_PRO_ACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cell 1 — Imports & setup for the ALS QNN notebook

**Purpose:** bring in the sklearn utilities, pin/install Qiskit, and import the quantum pieces used by a variational regressor.

### What’s in this cell
- **Data utilities:** `train_test_split`, `MinMaxScaler`
- **Metrics:** `mean_squared_error`, `pearsonr`
- **Install (pinned):** `%pip install qiskit~=1.0 qiskit-machine-learning~=0.8.1 qiskit_algorithms`
- **Quantum stack:** `ZZFeatureMap`, `RealAmplitudes`, `COBYLA`, `VQR`, `Sampler`

> **Note:** If the install line complains about `qiskit_algorithms`, the PyPI package name is often `qiskit-algorithms` (hyphen). We’re not changing your code here—this is just a heads-up.

### Why scale features?
Angle encoders work better when inputs live in a tight range; min–max scaling avoids angle wraparound and makes training steadier.

<details>
<summary>Quick I/O expectations</summary>
After preprocessing later:
- Feature vectors will be scaled to a small range (often [0, 1]).
- The `ZZFeatureMap(feature_dimension=...)` should match your final number of features.
</details>

---


In [ ]:
from sklearn.model_selection import train_test_split  # quick train/validation split
from sklearn.preprocessing import MinMaxScaler        # keep features in a compact range for angle encoding
from sklearn.metrics import mean_squared_error        # regression loss (lower is better)
from scipy.stats import pearsonr                      # correlation between predictions and targets (closer to 1 is better)
%pip install qiskit~=1.0 qiskit-machine-learning~=0.8.1 qiskit_algorithms  # pinned install; if it fails, try 'qiskit-algorithms' manually

# Qiskit Imports
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes  # feature map + ansatz for the variational circuit
from qiskit_algorithms.optimizers import COBYLA                  # gradient-free optimizer suited to noisy objectives
from qiskit_machine_learning.algorithms.regressors import VQR    # variational quantum regressor wrapper
from qiskit.primitives import Sampler                            # primitive that evaluates circuits (shot-based)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1


## Cell 2 — CV-safe PRO-ACT preprocessing (`ALSDataProcessor`)

**Purpose:** Turn the raw PRO-ACT CSVs into a clean, cross-validation-safe feature matrix `X` and target `y` (ALSFRS Total slope), then save a single file you can feed into any model (classical or quantum).

### What this cell does
- Anchors timelines to each subject’s **first ALSFRS visit** (t = 0).
- Builds features from the **first 0–90 days** after that anchor across longitudinal tables (ALSFRS, FVC, vitals, labs, grip, muscle).
- Harmonizes **ALSFRS-R** subitems when present (Q10 from 10a; merges Q5a/Q5b).
- Collapses **FVC trials** to the **max per test time** before summarizing.
- Creates **7 summaries** per signal: `min, max, median, std, first, last, slope`.
- Drops columns with **>30% missing**, but does **no scaling/encoding/imputation** here (to avoid leakage).
- Keeps only **eligible subjects**: have an ALSFRS measure **>3 months** and **>12 months** after anchor.
- Saves `final_processed_als_data.csv` and returns a dict with `X`, `y`, `subject_ids`, and the raw joined frame.

### Expected inputs (filenames)
- `PROACT_ALSFRS.csv` *(required)*, plus any of:  
  `PROACT_FVC.csv`, `PROACT_VITALSIGNS.csv`, `PROACT_RILUZOLE.csv`, `PROACT_DEMOGRAPHICS.csv`,  
  `PROACT_LABS.csv`, `PROACT_DEATHDATA.csv`, `PROACT_HANDGRIPSTRENGTH.csv`,  
  `PROACT_MUSCLESTRENGTH.csv`, `PROACT_ALSHISTORY.csv`.

> **Key columns the code looks for**
> - `subject_id` (auto-detected if a similar name exists)
> - Any time field containing **“delta”** or **“day”** (used as days since that table’s baseline)
> - In ALSFRS: `ALSFRS_Total` (numeric)

### Outputs
- **File:** `final_processed_als_data.csv`  
- **Return (dict):**  
  - `X` → features (after 30% missing filter)  
  - `y` → `alsfrs_slope` (per-subject target)  
  - `subject_ids` → subject mapping  
  - `raw_frame` → `[subject_id, alsfrs_slope, X...]` (what gets saved)

> **Windowing:** all longitudinal features are computed over **0–90 days** *from the ALSFRS anchor*.  
> **Slope target:** computed between the **first point after 3 months** and the **first point after 12 months**.

### Gotchas / tips
- If a table has **no time column**, it’s skipped with a warning.
- **Pivoting** of wide/long lab/grip/muscle tables is “best effort” by name patterns; if that fails, it continues safely.
- **No encodings** are done here. Do one-hot/ordinal and scaling **inside your CV folds**.
- If your time unit isn’t in days, ensure the column names include `day`/`delta` or rename before loading.

<details>
<summary>How features are built (quick recipe)</summary>

1) **Anchor** each subject’s rows to the first ALSFRS visit (t=0).  
2) **Align** other tables to ALSFRS time by subtracting each subject’s ALSFRS anchor day.  
3) **Restrict** to days **0–90**.  
4) For every numeric signal per table, compute:  
   `min, max, median, std (ddof=0), first, last, slope(first→last over time)`  
   *(slope is NaN if only one observation or zero time span).*  
5) **Merge** all per-table feature blocks on `subject_id`.
</details>

<details>
<summary>Eligibility logic for y</summary>
A subject is kept if they have **any** ALSFRS measure strictly **>3.0 months** *and* strictly **>12.0 months** after the anchor.  
The slope target uses the very first record after 3 months and the very first after 12 months.
</details>

---

In [ ]:
import pandas as pd
import numpy as np
import warnings
from typing import Dict, Optional, List

warnings.filterwarnings("ignore")  # keep notebook output tidy; data has mixed types/old columns


class ALSDataProcessor:
    """
    CV-safe preprocessing for PRO-ACT to reproduce the paper's EDA:
      - Anchor to FIRST ALSFRS visit (t=0) per subject
      - Inputs: first 3 months (0–90 days from anchor) for all longitudinal tables
      - Outcome: ALSFRS Total slope between FIRST-after-3mo and FIRST-after-12mo
      - ALSFRS-R harmonization hooks (Q10 from 10a; merge Q5a/Q5b)
      - FVC reduced to max-of-trials per test before summarization
      - Seven summaries: min, max, median, std, first, last, slope (slope=NaN if only 1 obs)
      - Drop features with >30% missing (no other transforms here — avoid leakage)
    """

    def __init__(self):
        # identifiers/time-like columns we should NOT summarize as numeric features
        self.id_and_delta_cols = {
            "subject_id",
            "alsfrs_delta",
            "fvc_delta",
            "vitals_delta",
            "labs_delta",
            "grip_delta",
            "muscle_delta",
            "onset_delta",
            "death_delta",
            "history_delta",
            "anchor_days",
            "days_from_alsfrs_anchor",
        }

    # --------- Utilities ---------

    @staticmethod
    def _find_time_col(df: pd.DataFrame) -> Optional[str]:
        """Find a time column that represents days since baseline in a table."""
        # Prefer delta
        for c in df.columns:
            lc = c.lower()
            if "delta" in lc:
                return c
        # Fallback to 'days' if present
        for c in df.columns:
            lc = c.lower()
            if "day" in lc:
                return c
        return None

    # --------- ALSFRS-R harmonization ---------

    def _convert_alsfrs_r(self, alsfrs_df: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare ALSFRS table. If ALSFRS-R subitems exist, map per paper:
          - Q10 <- 10a (dyspnea). Ignore 10b/10c.
          - Merge Q5a/Q5b into Q5 if present.
        If only totals exist, this is a no-op aside from coercions.
        """
        df = alsfrs_df.copy()

        if "ALSFRS_Total" in df.columns:
            df["ALSFRS_Total"] = pd.to_numeric(df["ALSFRS_Total"], errors="coerce")

        # Try to locate subitems by loose names
        cols = {c.lower(): c for c in df.columns}

        # Q10 from 10a (dyspnea)
        for candidate in ["alsfrs_r_q10a", "q10a", "dyspnea", "alsfrs_q10a"]:
            if candidate in cols:
                df["Q10"] = pd.to_numeric(df[cols[candidate]], errors="coerce")
                break

        # Merge Q5a/Q5b
        q5a = next(
            (cols[k] for k in ["alsfrs_r_q5a", "q5a", "cutting_wout_gastrostomy"] if k in cols),
            None,
        )
        q5b = next(
            (cols[k] for k in ["alsfrs_r_q5b", "q5b", "cutting_with_gastrostomy"] if k in cols),
            None,
        )
        if q5a and q5b:
            q5a_vals = pd.to_numeric(df[q5a], errors="coerce").values
            q5b_vals = pd.to_numeric(df[q5b], errors="coerce").values
            df["Q5"] = np.nanmax(np.vstack([q5a_vals, q5b_vals]), axis=0)

        return df

    # --------- Anchoring ---------

    def _alsfrs_anchor_days(self, alsfrs_df: pd.DataFrame) -> pd.Series:
        """
        Compute per-subject anchor day = first ALSFRS visit (min delta/days).
        """
        df = alsfrs_df.copy()
        tcol = self._find_time_col(df)
        if tcol is None:
            raise ValueError("ALSFRS table lacks a time delta/days column.")

        df.rename(columns={tcol: "alsfrs_delta"}, inplace=True)
        anchor_map = df.groupby("subject_id")["alsfrs_delta"].min()
        return anchor_map

    # --------- Data I/O ---------

    def load_and_inspect_data(self, file_path: str = "") -> Dict[str, pd.DataFrame]:
        datasets: Dict[str, pd.DataFrame] = {}
        file_list = [
            "PROACT_ALSFRS.csv",
            "PROACT_FVC.csv",
            "PROACT_VITALSIGNS.csv",
            "PROACT_RILUZOLE.csv",
            "PROACT_DEMOGRAPHICS.csv",
            "PROACT_LABS.csv",
            "PROACT_DEATHDATA.csv",
            "PROACT_HANDGRIPSTRENGTH.csv",
            "PROACT_MUSCLESTRENGTH.csv",
            "PROACT_ALSHISTORY.csv",
        ]
        print("--- Loading and Inspecting Data ---")
        for file_name in file_list:
            try:
                df = pd.read_csv(file_path + file_name, on_bad_lines="skip")
                # normalize subject_id
                if "subject_id" not in df.columns:
                    potential = [c for c in df.columns if "subject" in c.lower()]
                    if potential:
                        df = df.rename(columns={potential[0]: "subject_id"})
                # coerce delta-like numeric columns
                for c in df.columns:
                    if "delta" in c.lower() or "day" in c.lower():
                        df[c] = pd.to_numeric(df[c], errors="coerce")
                datasets[file_name] = df
                print(f"✓ {file_name}: {df.shape}")
            except FileNotFoundError:
                print(f"✗ {file_name}: File not found (skipped).")
        return datasets

    # --------- Outcome ---------

    def calculate_alsfrs_slope(self, alsfrs_df: pd.DataFrame) -> pd.DataFrame:
        """
        Outcome = slope between FIRST-after-3mo and FIRST-after-12mo ALSFRS totals,
        with time anchored to first ALSFRS visit.
        """
        df = alsfrs_df.copy()
        tcol = self._find_time_col(df)
        if tcol is None:
            raise ValueError("ALSFRS table lacks a time delta/days column.")
        if "ALSFRS_Total" not in df.columns:
            raise ValueError("ALSFRS_Total missing in ALSFRS table.")

        df.rename(columns={tcol: "alsfrs_delta"}, inplace=True)
        # Anchor
        anchor_map = df.groupby("subject_id")["alsfrs_delta"].min()
        df["days_from_anchor"] = df["alsfrs_delta"] - df["subject_id"].map(anchor_map)
        df["months"] = df["days_from_anchor"] / 30.44

        df = df.sort_values(["subject_id", "months"])
        slopes = {}

        for sid, g in df.groupby("subject_id", sort=False):
            g = g.dropna(subset=["months", "ALSFRS_Total"])
            t1 = g[g["months"] > 3.0].head(1)
            t2 = g[g["months"] > 12.0].head(1)
            if not t1.empty and not t2.empty:
                t1m = float(t1["months"].iloc[0])
                t2m = float(t2["months"].iloc[0])
                t1v = float(t1["ALSFRS_Total"].iloc[0])
                t2v = float(t2["ALSFRS_Total"].iloc[0])
                if t2m > t1m:
                    slopes[sid] = (t2v - t1v) / (t2m - t1m)

        return pd.DataFrame({"subject_id": list(slopes.keys()), "alsfrs_slope": list(slopes.values())})

    # --------- FVC collapse ---------

    @staticmethod
    def _fvc_collapse_trials(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
        """
        Reduce FVC per row/time to the max across trials before summarization.
        Tries to detect typical trial columns; falls back gracefully.
        """
        d = df.copy()
        # Find obvious trial columns
        trial_cols = [c for c in d.columns if "trial" in c.lower()]
        # Some datasets have explicit liters columns per trial name
        if trial_cols:
            d["FVC_Liters"] = pd.to_numeric(d[trial_cols].max(axis=1), errors="coerce")
            keep = ["subject_id", time_col, "FVC_Liters"]
            return d[keep]
        # Fallbacks: look for liters column names
        liter_like = [c for c in d.columns if "liter" in c.lower() or "fvc" in c.lower()]
        if liter_like:
            # If multiple, take row-wise max
            d["FVC_Liters"] = pd.to_numeric(d[liter_like].max(axis=1), errors="coerce")
            keep = ["subject_id", time_col, "FVC_Liters"]
            return d[keep]
        # Last resort: return as-is
        return d

    # --------- Longitudinal summarization ---------

    def create_longitudinal_features(self, df: pd.DataFrame, time_col: str, prefix: str) -> pd.DataFrame:
        """
        Create 7 summaries over [0, 90] days from ALSFRS anchor:
          min, max, median, std, first, last, slope(first→last)
        Slope remains NaN if only one observation or zero time span.
        """
        if time_col not in df.columns:
            return pd.DataFrame()

        d = df.copy()
        # Coerce numerics (but keep subject_id/time cols)
        for c in d.columns:
            if c not in {"subject_id", time_col}:
                d[c] = pd.to_numeric(d[c], errors="coerce")

        # Ensure window is 0..90 days from ALSFRS anchor (already anchored)
        d = d[(d[time_col] >= 0) & (d[time_col] <= 90)].copy()
        if d.empty:
            return pd.DataFrame()

        # Value columns (exclude identifiers/derived delta/time)
        val_cols = [
            c
            for c in d.select_dtypes(include=[np.number]).columns
            if c not in self.id_and_delta_cols and c not in {"subject_id", time_col}
        ]
        if not val_cols:
            return pd.DataFrame()

        out = []
        g = d.groupby("subject_id", as_index=True)
        for col in val_cols:
            agg = g[col].agg(["min", "max", "median", "first", "last"])
            std_ = g[col].std(ddof=0).rename("std")
            slope = g.apply(
                lambda x: (x[col].iloc[-1] - x[col].iloc[0]) / max(1e-9, (x[time_col].iloc[-1] - x[time_col].iloc[0]))
                if len(x) > 1 and (x[time_col].iloc[-1] - x[time_col].iloc[0]) > 0
                else np.nan
            ).rename("slope")
            feat = pd.concat([agg, std_, slope], axis=1)
            feat.columns = [f"{prefix}{col}_{cname}" for cname in feat.columns]
            out.append(feat)

        return pd.concat(out, axis=1).reset_index()

    # --------- Static table processing (no encoding here to avoid leakage) ---------

    @staticmethod
    def process_static_data(df: pd.DataFrame) -> pd.DataFrame:
        """
        CV-safe: DO NOT encode here. Just keep one row per subject.
        (Do categorical encoding in your modeling pipeline.)
        """
        if "subject_id" not in df.columns:
            return pd.DataFrame()
        # Keep first non-duplicated row per subject_id
        return df.drop_duplicates(subset=["subject_id"]).copy()

    # --------- Merge features ---------

    def merge_all_features(self, datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        if "PROACT_DEMOGRAPHICS.csv" not in datasets:
            raise ValueError("Demographics file is missing.")

        # Build ALSFRS anchor map
        alsfrs = datasets["PROACT_ALSFRS.csv"]
        anchor_map = self._alsfrs_anchor_days(alsfrs)

        # Start with demographics (static)
        final_df = self.process_static_data(datasets["PROACT_DEMOGRAPHICS.csv"])

        # Add static-ish other tables (keep CV-safe; no encodings)
        for file in ["PROACT_RILUZOLE.csv", "PROACT_ALSHISTORY.csv"]:
            if file in datasets:
                static_df = self.process_static_data(datasets[file])
                final_df = pd.merge(final_df, static_df, on="subject_id", how="left")

        # Longitudinal configs
        longitudinal = {
            "PROACT_ALSFRS.csv": "alsfrs_",
            "PROACT_FVC.csv": "fvc_",
            "PROACT_VITALSIGNS.csv": "vitals_",
            "PROACT_LABS.csv": "labs_",
            "PROACT_HANDGRIPSTRENGTH.csv": "grip_",
            "PROACT_MUSCLESTRENGTH.csv": "muscle_",
        }

        print("\n--- Generating Longitudinal Features (anchored to first ALSFRS; window = 0–90 days) ---")
        for file, prefix in longitudinal.items():
            if file not in datasets:
                continue

            df = datasets[file].copy()
            tcol = self._find_time_col(df)
            if tcol is None:
                print(f"Warning: No time delta/days column in {file}. Skipping.")
                continue

            # Anchor this table to ALSFRS first visit
            df["anchor_days"] = df["subject_id"].map(anchor_map)
            df = df[~df["anchor_days"].isna()].copy()
            df["days_from_alsfrs_anchor"] = pd.to_numeric(df[tcol], errors="coerce") - df["anchor_days"]

            # FVC special handling: collapse to max-of-trials BEFORE summarization
            if file == "PROACT_FVC.csv":
                df = self._fvc_collapse_trials(df, time_col="days_from_alsfrs_anchor")

            # Attempt to pivot long-form measurement tables (best effort)
            if file in {"PROACT_LABS.csv", "PROACT_MUSCLESTRENGTH.csv", "PROACT_HANDGRIPSTRENGTH.csv"}:
                try:
                    test_cols = [
                        c
                        for c in df.columns
                        if c not in {"subject_id", "days_from_alsfrs_anchor", "anchor_days"}
                        and any(k in c.lower() for k in ["test", "exam", "muscle", "site", "name", "strength_test"])
                    ]
                    value_cols = [
                        c
                        for c in df.columns
                        if c not in {"subject_id", "days_from_alsfrs_anchor", "anchor_days"}
                        and any(k in c.lower() for k in ["result", "value", "strength", "score"])
                    ]
                    if test_cols and value_cols:
                        tcol_name = test_cols[0]
                        vcol_name = value_cols[0]
                        df[vcol_name] = pd.to_numeric(df[vcol_name], errors="coerce")
                        df = (
                            df.pivot_table(
                                index=["subject_id", "days_from_alsfrs_anchor"],
                                columns=tcol_name,
                                values=vcol_name,
                                aggfunc="mean",
                            )
                            .reset_index()
                        )
                except Exception as e:
                    print(f"Warning: Pivoting failed for {file}: {e}")

            feats = self.create_longitudinal_features(df, "days_from_alsfrs_anchor", prefix)
            if not feats.empty:
                final_df = pd.merge(final_df, feats, on="subject_id", how="left")

        return final_df

    # --------- Eligibility ---------

    def filter_eligible_patients(self, feature_df: pd.DataFrame, alsfrs_df: pd.DataFrame) -> pd.DataFrame:
        """
        Keep subjects who have ANY ALSFRS >3 months AND >12 months AFTER the ALSFRS anchor.
        """
        df = alsfrs_df.copy()
        tcol = self._find_time_col(df)
        if tcol is None:
            raise ValueError("ALSFRS table lacks a time delta/days column.")

        df.rename(columns={tcol: "alsfrs_delta"}, inplace=True)
        anchor_map = df.groupby("subject_id")["alsfrs_delta"].min()
        df["days_from_anchor"] = df["alsfrs_delta"] - df["subject_id"].map(anchor_map)
        df["months"] = df["days_from_anchor"] / 30.44

        g = df.groupby("subject_id")["months"]
        has_t1 = g.apply(lambda s: (s > 3.0).any())
        has_t2 = g.apply(lambda s: (s > 12.0).any())
        eligible_ids = has_t1[has_t1].index.intersection(has_t2[has_t2].index)

        print(f"\nEligible patients: {len(eligible_ids)} / {df['subject_id'].nunique()}")
        return feature_df[feature_df["subject_id"].isin(eligible_ids)].copy()

    # --------- Orchestration ---------

    def run_pipeline(self, file_path: str = "") -> Optional[Dict[str, pd.DataFrame]]:
        """
        End-to-end EDA (CV-safe) that writes 'final_processed_als_data.csv'.
        No imputation/scaling/feature selection here — do that inside your CV pipeline.
        """
        print("====== Starting ALS Data Preprocessing Pipeline ======")
        datasets = self.load_and_inspect_data(file_path)
        if "PROACT_ALSFRS.csv" not in datasets:
            print("CRITICAL ERROR: PROACT_ALSFRS.csv not found. Aborting.")
            return None

        # ALSFRS prep + anchor
        datasets["PROACT_ALSFRS.csv"] = self._convert_alsfrs_r(datasets["PROACT_ALSFRS.csv"])

        # Outcome
        target_df = self.calculate_alsfrs_slope(datasets["PROACT_ALSFRS.csv"])
        print(f"\nCalculated ALSFRS slope for {len(target_df)} patients.")

        # Features
        full_features = self.merge_all_features(datasets)

        # Eligibility
        eligible_features = self.filter_eligible_patients(full_features, datasets["PROACT_ALSFRS.csv"])

        # Join features + target
        final_df = pd.merge(eligible_features, target_df, on="subject_id", how="inner")

        # Drop features with >30% missing
        print("\n--- Handling Missing Values (Dropping cols with >30% missing) ---")
        initial_cols = len(final_df.columns)
        missing_thresh = 0.30
        min_non_na = int(np.ceil(len(final_df) * (1 - missing_thresh)))
        final_df = final_df.dropna(axis=1, thresh=min_non_na)
        dropped = initial_cols - len(final_df.columns)
        print(f"Dropped {dropped} columns for >{int(missing_thresh*100)}% missingness.")

        # Separate X/y (no transforms here to avoid leakage)
        if "alsfrs_slope" not in final_df.columns:
            print("No target available after merges. Aborting.")
            return None

        y = final_df["alsfrs_slope"]
        valid = y.notna()
        final_df = final_df.loc[valid].reset_index(drop=True)

        subject_ids = final_df["subject_id"]
        y = final_df["alsfrs_slope"]
        X = final_df.drop(columns=["subject_id", "alsfrs_slope"])

        # Save CV-safe engineered dataset (raw features)
        out = pd.concat([subject_ids, y, X], axis=1)
        out.to_csv("final_processed_als_data.csv", index=False)
        print("\n✅ Saved CV-safe engineered data to 'final_processed_als_data.csv'")
        print(f"Feature matrix shape: {X.shape} | Target length: {len(y)}")

        return {"X": X, "y": y, "subject_ids": subject_ids, "raw_frame": out}


if __name__ == "__main__":
    # If your CSVs live elsewhere, set file_path accordingly (e.g., "C:/data/PROACT/")
    file_path = ""
    processor = ALSDataProcessor()
    processed = processor.run_pipeline(file_path=file_path)
    if processed is not None:
        print("\nPreview of columns:", list(processed["X"].columns)[:10])
        print("Done.")


====== Starting ALS Data Preprocessing Pipeline ======
--- Loading and Inspecting Data ---
✓ PROACT_ALSFRS.csv: (73845, 20)
✓ PROACT_FVC.csv: (49110, 10)
✓ PROACT_VITALSIGNS.csv: (84721, 36)
✓ PROACT_RILUZOLE.csv: (10363, 3)
✓ PROACT_DEMOGRAPHICS.csv: (12504, 14)
✓ PROACT_LABS.csv: (2937162, 5)
✓ PROACT_DEATHDATA.csv: (5043, 3)
✓ PROACT_HANDGRIPSTRENGTH.csv: (19032, 11)
✓ PROACT_MUSCLESTRENGTH.csv: (204875, 10)
✓ PROACT_ALSHISTORY.csv: (13765, 16)

Calculated ALSFRS slope for 1897 patients.

--- Generating Longitudinal Features (anchored to first ALSFRS; window = 0–90 days) ---

Eligible patients: 3317 / 8538

--- Handling Missing Values (Dropping cols with >30% missing) ---
Dropped 1413 columns for >30% missingness.

✅ Saved CV-safe engineered data to 'final_processed_als_data.csv'
Feature matrix shape: (1897, 346) | Target length: 1897

Preview of columns: ['Demographics_Delta', 'Age', 'Race_Caucasian', 'Sex', 'Subject_used_Riluzole', 'Riluzole_use_Delta', 'Subject_ALS_History_Delta'

## Cell 5 — FAST classical baselines (successive halving + pipeline caching)

**Purpose:** Train strong-but-quick RF & SVR baselines using **successive halving** (prunes weak configs early) and **joblib cache** (reuses preprocessing across CV folds).

### What this cell does
- Loads `final_processed_als_data.csv`, quick **80/20** split, optional shuffle for fold homogeneity.
- Auto-detects **numeric vs categorical** columns.
- Builds **in-pipeline** preprocessing (impute; scaling only for SVR; OHE for categoricals).
- Runs **HalvingGridSearchCV (cv=3)** for:
  - **RandomForestRegressor** (no scaling needed).
  - **SVR (RBF)** (with scaling).
- Evaluates on the held-out test set and prints **RMSE, PCC** with **95% bootstrap CIs**.
- Prints a quick **RF+SVR 50–50 ensemble** as a sanity check.

### Why it’s fast
- **Successive halving** cuts off underperformers early → far fewer total fits.
- **joblib.Memory** caches transformers inside the pipeline → repeated CV splits don’t recompute imputation/encoding from scratch.

### Outputs (printed)
- Best params for RF & SVR (FAST grids).
- **Test Set Performance (FAST mode)** table with RMSE & PCC + CIs.
- Optional **RF+SVR Ensemble** metrics.

> Tip: If you’re only benchmarking speed, comment out the SVR block; RF alone is often a strong baseline here.

---


In [ ]:
import numpy as np
import pandas as pd
from typing import Tuple, Dict
import warnings
warnings.filterwarnings("ignore")

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

# Halving search (successive halving)
from sklearn.experimental import enable_halving_search_cv  # noqa: F401
from sklearn.model_selection import HalvingGridSearchCV

# caching
from joblib import Memory

np.random.seed(42)

# ---------- Metrics ----------
def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def safe_pcc(y_true, y_pred) -> float:
    yt = np.asarray(y_true, dtype=float).ravel()
    yp = np.asarray(y_pred, dtype=float).ravel()
    if yt.std() < 1e-12 or yp.std() < 1e-12:
        return 0.0
    return float(np.corrcoef(yt, yp)[0, 1])

def bootstrap_ci(y_true, y_pred, metric_fn, n_boot=800, alpha=0.95, seed=42) -> Tuple[float, float]:
    rng = np.random.default_rng(seed)
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    n = len(y_true)
    stats = []
    idx = np.arange(n)
    for _ in range(n_boot):
        b = rng.choice(idx, size=n, replace=True)
        stats.append(metric_fn(y_true[b], y_pred[b]))
    lo = float(np.percentile(stats, (1 - alpha) / 2 * 100))
    hi = float(np.percentile(stats, (1 + alpha) / 2 * 100))
    return lo, hi

# ---------- Main ----------
def run_classical_pipeline_fast() -> pd.DataFrame:
    print("====== FAST Classical Baselines (successive halving, cached) ======")

    # 1) Load engineered data
    df = pd.read_csv("final_processed_als_data.csv")
    print(f"✓ Loaded engineered dataset: {df.shape}")

    X = df.drop(columns=["subject_id", "alsfrs_slope"])
    y = df["alsfrs_slope"].astype(float)

    # Optional quick shuffle for better fold homogeneity
    X, y = shuffle(X, y, random_state=42)

    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42
    )
    print(f"Split: train={X_train.shape[0]}, test={X_test.shape[0]}")

    # 2) Column typing
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
    print(f"Detected numeric={len(num_cols)}, categorical={len(cat_cols)}")

    # Pipeline cache
    memory = Memory(location="sk_cache", verbose=0)

    # Preprocessors
    # Numeric: impute → (optional scaler in SVR branch)
    num_rf = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ], memory=memory)

    num_svr = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ], memory=memory)

    if len(cat_cols) > 0:
        cat_common = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
        ], memory=memory)
        preproc_rf = ColumnTransformer(
            transformers=[("num", num_rf, num_cols), ("cat", cat_common, cat_cols)],
            remainder="drop"
        )
        preproc_svr = ColumnTransformer(
            transformers=[("num", num_svr, num_cols), ("cat", cat_common, cat_cols)],
            remainder="drop"
        )
    else:
        # No categoricals → simpler (faster) preprocessors
        preproc_rf = num_rf
        preproc_svr = num_svr

    # 3) Pipelines (with small but effective grids)
    rf_pipe = Pipeline(steps=[
        ("preprocess", preproc_rf),
        ("select", SelectKBest(score_func=f_regression, k="all")),
        ("model", RandomForestRegressor(random_state=42, n_jobs=-1))
    ], memory=memory)

    rf_grid: Dict[str, list] = {
        # keep imputer fixed (median) to avoid recomputing transforms
        "select__k": ["all", 50],              # feature count toggle
        "model__n_estimators": [250],          # enough trees, faster than 500
        "model__max_depth": [None, 12],
        "model__min_samples_leaf": [1, 2],
        "model__max_features": ["sqrt"],       # stable setting
    }

    # Successive halving (aggressive elimination reduces fits)
    rf_search = HalvingGridSearchCV(
        rf_pipe,
        rf_grid,
        factor=3,
        resource="n_samples",
        min_resources="exhaust",
        cv=3,
        scoring="neg_root_mean_squared_error",   # optimize RMSE directly
        n_jobs=-1,
        verbose=0,
        refit=True
    )
    print("\n--- Fitting RandomForest (HalvingGridSearchCV, cv=3) ---")
    rf_search.fit(X_train, y_train)
    print(f"RF best params: {rf_search.best_params_}")

    # SVR (trimmed grid; if you need even faster, comment this whole block)
    svr_pipe = Pipeline(steps=[
        ("preprocess", preproc_svr),
        ("select", SelectKBest(score_func=f_regression, k="all")),
        ("model", SVR(kernel="rbf"))
    ], memory=memory)

    svr_grid: Dict[str, list] = {
        "select__k": ["all", 50],
        "model__C": [1.0, 3.0],
        "model__epsilon": [0.1],
        "model__gamma": ["scale"],
    }

    svr_search = HalvingGridSearchCV(
        svr_pipe,
        svr_grid,
        factor=3,
        resource="n_samples",
        min_resources="exhaust",
        cv=3,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1,
        verbose=0,
        refit=True
    )
    print("\n--- Fitting SVR (HalvingGridSearchCV, cv=3) ---")
    svr_search.fit(X_train, y_train)
    print(f"SVR best params: {svr_search.best_params_}")

    # 4) Test-set evaluation + (faster) bootstrap CIs
    results = []

    for name, est in [("Random Forest", rf_search), ("SVR (RBF)", svr_search)]:
        y_pred = est.best_estimator_.predict(X_test)
        test_rmse = rmse(y_test, y_pred)
        test_pcc  = safe_pcc(y_test.values, y_pred)

        rmse_lo, rmse_hi = bootstrap_ci(y_test.values, y_pred, rmse, n_boot=800, alpha=0.95, seed=123)
        pcc_lo,  pcc_hi  = bootstrap_ci(y_test.values, y_pred, safe_pcc, n_boot=800, alpha=0.95, seed=456)

        results.append({
            "Model": name,
            "RMSE": test_rmse,
            "RMSE 95% CI Low": rmse_lo,
            "RMSE 95% CI High": rmse_hi,
            "PCC": test_pcc,
            "PCC 95% CI Low": pcc_lo,
            "PCC 95% CI High": pcc_hi,
        })

    results_df = pd.DataFrame(results).set_index("Model")
    print("\n====== Test Set Performance (FAST mode) ======")
    print(results_df.round(4))

    # Optional quick 50–50 blend (no extra CV)
    rf_pred = rf_search.best_estimator_.predict(X_test)
    svr_pred = svr_search.best_estimator_.predict(X_test)
    ens_pred = 0.5 * (rf_pred + svr_pred)

    ens_rmse = rmse(y_test, ens_pred)
    ens_pcc  = safe_pcc(y_test.values, ens_pred)
    ens_rmse_ci = bootstrap_ci(y_test.values, ens_pred, rmse, n_boot=800, alpha=0.95, seed=789)
    ens_pcc_ci  = bootstrap_ci(y_test.values, ens_pred, safe_pcc, n_boot=800, alpha=0.95, seed=101112)

    print("\n--- Simple RF+SVR Avg Ensemble (FAST) ---")
    print(pd.DataFrame({
        "RMSE": [ens_rmse],
        "RMSE 95% CI Low": [ens_rmse_ci[0]],
        "RMSE 95% CI High": [ens_rmse_ci[1]],
        "PCC": [ens_pcc],
        "PCC 95% CI Low": [ens_pcc_ci[0]],
        "PCC 95% CI High": [ens_pcc_ci[1]],
    }, index=["RF+SVR Ensemble"]).round(4))

    return results_df


if __name__ == "__main__":
    run_classical_pipeline_fast()


====== FAST Classical Baselines (successive halving, cached) ======
✓ Loaded engineered dataset: (1897, 348)
Split: train=1517, test=380
Detected numeric=343, categorical=3

--- Fitting RandomForest (HalvingGridSearchCV, cv=3) ---
RF best params: {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 2, 'model__n_estimators': 250, 'select__k': 50}

--- Fitting SVR (HalvingGridSearchCV, cv=3) ---
SVR best params: {'model__C': 1.0, 'model__epsilon': 0.1, 'model__gamma': 'scale', 'select__k': 'all'}

====== Test Set Performance (FAST mode) ======
                 RMSE  RMSE 95% CI Low  RMSE 95% CI High     PCC  \
Model                                                              
Random Forest  0.5905           0.5467            0.6386  0.1918   
SVR (RBF)      0.5907           0.5405            0.6404  0.2131   

               PCC 95% CI Low  PCC 95% CI High  
Model                                           
Random Forest          0.0909           0.2972  


## Cell 4 — Variational Quantum Regressor (VQC) trained with SPSA

**Purpose:** End-to-end quantum regressor that predicts ALSFRS slope from engineered features.  
It builds a circuit **ZZFeatureMap → EfficientSU2**, measures the **average Z** observable, and fits a tiny linear head `ŷ = α·⟨O⟩ + β`. Parameters **θ (circuit), α, β** are trained with **SPSA** on MSE using an Aer/Estimator backend.

### Pipeline (quick map)
1) **Load** `final_processed_als_data.csv` → keep **numeric** columns only.  
2) **Train/test split** (80/20).  
3) **Feature pick** inside train: **RF top-K** numeric features (default **K=16**).  
4) **Train/val split** within training set for **early stopping**.  
5) **Impute + Standardize** (train-only fit) → **PLSRegression** → reduce to `n_qubits`.  
6) **Min-max to angles** `[0, π]` per component (angle encoding).  
7) **Circuit**: `ZZFeatureMap(n_qubits, reps) ∘ EfficientSU2(n_qubits, reps, entanglement="linear")`.  
8) **Observable**: mean of Z on each qubit: \( O = \frac{1}{n}\sum_i Z_i \).  
9) **Train with SPSA** on MSE, early stop on `val_rmse + (1 - val_pcc)`.  
10) **Test**: RMSE, PCC + **95% bootstrap CIs**.

### Key knobs (you can pass via `run_vqc`)
- `n_qubits` (e.g., 4) & `pls_components` (**must match** `n_qubits`).
- `topk` features before PLS (default 16).
- Circuit depth: `fmap_reps`, `ansatz_reps`.
- SPSA schedule: `a, c, A, alpha, gamma`; steps & `batch_size`.
- Early stopping `patience`. Backend auto-selects **AerEstimator** if available.

> **Tip:** For faster smoke tests, try `spsa_steps=150` and `batch_size=64`.

<details>
<summary>SPSA in 30 seconds</summary>
At step *k*, build **one random ± perturbation** of the whole parameter vector, compute two losses `L+` and `L−`, estimate a stochastic gradient with a finite difference, and apply a scheduled step size. It needs **2 objective calls per step**, independent of parameter count.
</details>

<details>
<summary>Encoding & Observable</summary>
- **Encoding:** PLS projects scaled features into `n_qubits` components, then min-max maps them into **[0, π]** angles to feed the `ZZFeatureMap`.  
- **Observable:** ⟨O⟩ is between −1 and 1; the linear head `(α, β)` rescales it to the target range.
</details>

<details>
<summary>Gotchas</summary>
- **`pls_components == n_qubits`** is asserted.  
- If **qiskit-aer** isn’t present, it falls back to the built-in `Estimator` (slower).  
- `topk` must be ≤ number of numeric features in the training split.  
- Seeds are set for numpy and RF; quantum backends may still introduce sampling noise.
</details>

**Printed outputs:** split sizes, top-K list, live train/val metrics during SPSA, and final test **RMSE/PCC + 95% CIs**.

---


In [ ]:
# vqc_multiobs_spsa_ckpt.py
# End-to-end variational quantum regressor with multi-observable readout (no classical model on QRF).
# Trains θ (circuit) + w (readout vector) + b with SPSA.
# Includes checkpointing (best + latest) and resume support for Colab robustness.
# Evaluates on 80/20 split with RMSE/PCC + 95% CIs (paper-style).

import os, time, numpy as np, pandas as pd
from typing import Tuple, List, Optional, Dict
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

# keep CPU libs tame & reproducible
for k in ["OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS","NUMEXPR_NUM_THREADS"]:
    os.environ.setdefault(k, "1")
np.random.seed(42)

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# stats
from scipy.stats import pearsonr

# qiskit
from qiskit.circuit.library import ZZFeatureMap, EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
try:
    from qiskit_aer.primitives import Estimator as AerEstimator
    AER_OK = True
except Exception:
    AER_OK = False


# -------------------- utilities --------------------
def safe_pcc(a, b):
    a, b = np.asarray(a).ravel(), np.asarray(b).ravel()
    if a.std() == 0 or b.std() == 0:
        return 0.0
    v = pearsonr(a, b)[0]
    return float(v) if np.isfinite(v) else 0.0

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def bootstrap_ci(y_true, y_pred, metric_fn, n_boot=5000, alpha=0.95, seed=42) -> Tuple[float, float]:
    rng = np.random.default_rng(seed)
    n = len(y_true); idx = np.arange(n)
    vals = []
    for _ in range(n_boot):
        b = rng.choice(idx, size=n, replace=True)
        vals.append(metric_fn(y_true[b], y_pred[b]))
    lo, hi = np.percentile(vals, [(1-alpha)/2*100, (1+alpha)/2*100])
    return float(lo), float(hi)

def zscore_fit(y):
    mu, sd = float(np.mean(y)), float(np.std(y) + 1e-12)
    return mu, sd

def zscore_apply(y, mu, sd):
    return (y - mu) / sd

def zscore_invert(yz, mu, sd):
    return yz * sd + mu

def load_numeric_xy(path="final_processed_als_data.csv"):
    df = pd.read_csv(path)
    X_all = df.drop(columns=["subject_id","alsfrs_slope"], errors="ignore")
    X = X_all.select_dtypes(include=[np.number]).copy()
    y = df["alsfrs_slope"].astype(float).values
    m = ~np.isnan(y)
    X, y = X.loc[m].reset_index(drop=True), y[m]
    print(f"✓ Data: X_num={X.shape} (from {X_all.shape[1]} total), y={y.shape}")
    return X, y

def select_topk_by_rf(X_df, y, k=16):
    imp = SimpleImputer(strategy="median")
    Xn = imp.fit_transform(X_df)
    rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1).fit(Xn, y)
    idx = np.argsort(rf.feature_importances_)[::-1][:k]
    cols = [X_df.columns[i] for i in idx]
    print(f"✓ Top-{k} numeric features: {cols}")
    return idx, cols

def _idx_from_param_name(name: str) -> int:
    if '[' in name and ']' in name:
        return int(name.split('[')[1].split(']')[0])
    if '_' in name:
        tail = name.split('_')[-1]
        if tail.isdigit(): return int(tail)
    digits = ''.join(ch for ch in name if ch.isdigit())
    return int(digits) if digits else 0

def ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def ckpt_paths(ckpt_dir: str, seed: int) -> Dict[str, str]:
    ensure_dir(ckpt_dir)
    latest = os.path.join(ckpt_dir, f"seed-{seed}-latest.npz")
    best   = os.path.join(ckpt_dir, f"seed-{seed}-best.npz")
    return {"latest": latest, "best": best}

def save_ckpt(path: str, step: int, theta: np.ndarray, w: np.ndarray, b: float,
              best_metric: float, extras: Optional[dict] = None):
    payload = {
        "step": np.array(step, dtype=np.int64),
        "theta": np.array(theta, dtype=np.float64),
        "w": np.array(w, dtype=np.float64),
        "b": np.array(b, dtype=np.float64),
        "best_metric": np.array(best_metric, dtype=np.float64),
    }
    if extras:
        for k, v in extras.items():
            try:
                payload[k] = np.array(v)
            except Exception:
                pass
    np.savez(path, **payload)

def load_ckpt(path: str):
    data = np.load(path, allow_pickle=True)
    step = int(data["step"])
    theta = np.array(data["theta"], dtype=np.float64)
    w = np.array(data["w"], dtype=np.float64)
    b = float(np.array(data["b"], dtype=np.float64))
    best_metric = float(np.array(data["best_metric"], dtype=np.float64))
    return step, theta, w, b, best_metric


# -------------------- observables --------------------
def make_observables(n_qubits=4, basis="Z", use_pairs=True):
    """
    basis:
      'Z'   -> {Z_i} + (optional) {Z_i Z_j}
      'ZX'  -> {Z_i, X_i} + (optional) {Z_i Z_j}
      'ZXY' -> {Z_i, X_i, Y_i} + (optional) {Z_i Z_j}
    Returns list[SparsePauliOp].
    """
    obs: List[SparsePauliOp] = []
    for i in range(n_qubits):
        p = ['I']*n_qubits; p[i] = 'Z'
        obs.append(SparsePauliOp.from_list([("".join(p[::-1]), 1.0)]))
        if basis in ("ZX","ZXY"):
            p = ['I']*n_qubits; p[i] = 'X'
            obs.append(SparsePauliOp.from_list([("".join(p[::-1]), 1.0)]))
        if basis == "ZXY":
            p = ['I']*n_qubits; p[i] = 'Y'
            obs.append(SparsePauliOp.from_list([("".join(p[::-1]), 1.0)]))
    if use_pairs:
        for i in range(n_qubits):
            for j in range(i+1, n_qubits):
                p = ['I']*n_qubits; p[i] = p[j] = 'Z'
                obs.append(SparsePauliOp.from_list([("".join(p[::-1]), 1.0)]))
    return obs


# -------------------- VQC with multi-observable readout + checkpointing --------------------
class VQRegressorSPSA_MultiObs:
    """
    y_hat = w^T E(x; θ) + b,   where E returns expectations of a set of observables.
    Trains θ (variational circuit), w (readout weights), b (bias) jointly with SPSA.
    """
    def __init__(self, n_qubits=4, fmap_reps=2, ansatz_reps=2, basis="Z", use_pairs=True,
                 estimator=None, seed=13, l2=0.0):
        self.nq = n_qubits
        self.fmap = ZZFeatureMap(feature_dimension=n_qubits, reps=fmap_reps)
        self.ans  = EfficientSU2(num_qubits=n_qubits, reps=ansatz_reps, entanglement="linear")
        self.circ = self.fmap.compose(self.ans)

        # variational params & feature params
        self.theta_params = sorted(list(self.ans.parameters), key=lambda p: _idx_from_param_name(p.name))
        self.x_params = sorted([p for p in self.circ.parameters if p.name.startswith("x")],
                               key=lambda p: _idx_from_param_name(p.name))
        assert len(self.x_params) == self.nq, "Feature parameters mismatch"
        self.p = len(self.theta_params)

        # observables for readout
        self.obs_list: List[SparsePauliOp] = make_observables(n_qubits, basis=basis, use_pairs=use_pairs)
        self.D = len(self.obs_list)

        # estimator
        self.est = estimator if estimator is not None else (AerEstimator() if AER_OK else Estimator())

        # parameters (set on fit)
        self.theta = None                 # shape [p]
        self.w = None                     # shape [D]
        self.b = None                     # scalar
        self.l2 = float(l2)               # L2 penalty on w

    def _bind_circuits_batch(self, X_theta: np.ndarray, theta_vec: np.ndarray):
        circuits = []
        bind_theta = {self.theta_params[i]: float(theta_vec[i]) for i in range(self.p)}
        for i in range(X_theta.shape[0]):
            bind_feats = {self.x_params[k]: float(X_theta[i, k]) for k in range(self.nq)}
            cb = self.circ.assign_parameters({**bind_theta, **bind_feats}, inplace=False)
            circuits.append(cb)
        return circuits

    def _expectations_multi(self, X_theta: np.ndarray, theta_vec: np.ndarray, batch=128) -> np.ndarray:
        """
        Return matrix E in R^{N x D} of expectation values per observable.
        """
        N = X_theta.shape[0]; D = self.D
        out = np.empty((N, D), dtype=float)
        for s in range(0, N, batch):
            e = min(N, s+batch)
            circs = self._bind_circuits_batch(X_theta[s:e], theta_vec)
            # evaluate each sample against all observables
            circuits = []; obs_all = []
            for cb in circs:
                circuits.extend([cb]*D)
                obs_all.extend(self.obs_list)
            vals = self.est.run(circuits, obs_all).result().values
            out[s:e] = np.asarray(vals, dtype=float).reshape(e-s, D)
        return out

    def predict_z(self, X_theta: np.ndarray) -> np.ndarray:
        """
        Predict in z-scored target space (since training minimizes MSE there).
        """
        E = self._expectations_multi(X_theta, self.theta)
        return E @ self.w + self.b

    def fit(self, X_tr_th: np.ndarray, yz_tr: np.ndarray,
            X_val_th: np.ndarray=None, yz_val: np.ndarray=None,
            steps=800, batch_size=160,
            a=0.12, c=0.15, A=50, alpha=0.602, gamma=0.101,
            theta_init=None, w_init=None, b_init=None,
            seed=13, verbose=True, patience=100,
            # checkpointing
            checkpoint_dir: str = "checkpoints_vqc",
            checkpoint_every: int = 50,
            resume: bool = True):
        """
        SPSA on params P = [theta (p), w (D), b (1)].
        Loss = MSE(ŷ, y) + l2 * ||w||^2 / D.
        Early stopping on val objective: RMSE + (1-PCC).
        Saves 'latest' every `checkpoint_every` steps and 'best' on improvement.
        If resume and latest checkpoint exists, loads and continues from next step.
        """
        rng = np.random.default_rng(seed)
        N = len(yz_tr)
        cp = ckpt_paths(checkpoint_dir, seed)

        start_step = 1
        best_val = np.inf
        # ----- resume if allowed -----
        if resume and os.path.exists(cp["latest"]):
            try:
                s, th, wv, bv, best_metric = load_ckpt(cp["latest"])
                self.theta, self.w, self.b = th, wv, bv
                best_val = best_metric
                start_step = int(s) + 1
                if verbose:
                    tqdm.write(f"[Resume] Loaded latest checkpoint at step {s} (best_metric={best_metric:.6f}).")
            except Exception as e:
                if verbose:
                    tqdm.write(f"[Resume] Failed to load latest checkpoint: {e}. Starting fresh.")

        # ----- fresh init if needed -----
        if self.theta is None:
            if theta_init is None:
                theta_init = rng.normal(0, 0.25, size=self.p)
            self.theta = theta_init.astype(float)

        if self.w is None or self.b is None:
            if w_init is None:
                # quick LS init for (w,b) on a small subset with current theta
                idx0 = np.arange(min(256, N))
                E0 = self._expectations_multi(X_tr_th[idx0], self.theta, batch=min(batch_size, 128))
                y0 = yz_tr[idx0]
                A_ = np.c_[E0, np.ones(len(idx0))]
                wls, *_ = np.linalg.lstsq(A_, y0, rcond=None)
                self.w = wls[:-1]
                self.b = float(wls[-1])
            else:
                self.w = w_init.astype(float)
                self.b = float(b_init if b_init is not None else 0.0)

        if best_val is np.inf:
            # initialize best_val from current validation if available
            if (X_val_th is not None) and (yz_val is not None):
                yv_hat = self.predict_z(X_val_th)
                va_rm, va_pc = rmse(yz_val, yv_hat), safe_pcc(yz_val, yv_hat)
                best_val = va_rm + (1.0 - va_pc)

        hist = []
        dim = self.p + self.D + 1

        try:
            for k in range(start_step, steps+1):
                # SPSA gains
                ak = a / ((k + A)**alpha)
                ck = c / (k**gamma)

                # minibatch
                mb = min(batch_size, N)
                mb_idx = rng.choice(np.arange(N), size=mb, replace=False)
                Xb, yb = X_tr_th[mb_idx], yz_tr[mb_idx]

                delta = rng.choice([-1.0, 1.0], size=dim)

                # unpack deltas
                d_theta = delta[:self.p]
                d_w     = delta[self.p:self.p+self.D]
                d_b     = delta[-1]

                # plus/minus params
                th_plus,  th_minus  = self.theta + ck*d_theta, self.theta - ck*d_theta
                w_plus,   w_minus   = self.w     + ck*d_w,     self.w     - ck*d_w
                b_plus,   b_minus   = self.b     + ck*d_b,     self.b     - ck*d_b

                # expectations
                Eb = self._expectations_multi(Xb, th_plus,  batch=mb)
                Fb = self._expectations_multi(Xb, th_minus, batch=mb)

                # predictions in z-space
                y_plus  = Eb @ w_plus + b_plus
                y_minus = Fb @ w_minus + b_minus

                # MSE + L2(w)
                L_plus  = np.mean((y_plus  - yb)**2) + self.l2 * (np.sum(w_plus**2) / self.D)
                L_minus = np.mean((y_minus - yb)**2) + self.l2 * (np.sum(w_minus**2) / self.D)

                # gradient estimate
                ghat = (L_plus - L_minus) / (2.0 * ck) * (1.0 / delta)

                # slice gradients
                g_theta = ghat[:self.p]
                g_w     = ghat[self.p:self.p+self.D]
                g_b     = ghat[-1]

                # update
                self.theta = self.theta - ak * g_theta
                self.w     = self.w     - ak * g_w
                self.b     = self.b     - ak * g_b

                # monitor + early stopping + checkpointing
                do_print = (k % 10 == 0) or (k == 1)
                if do_print or (k % checkpoint_every == 0):
                    with np.errstate(all="ignore"):
                        ytr_hat = self.predict_z(X_tr_th)
                        tr_rm, tr_pc = rmse(yz_tr, ytr_hat), safe_pcc(yz_tr, ytr_hat)

                        if X_val_th is not None:
                            yv_hat = self.predict_z(X_val_th)
                            va_rm, va_pc = rmse(yz_val, yv_hat), safe_pcc(yz_val, yv_hat)
                            obj = va_rm + (1.0 - va_pc)
                            hist.append((k, tr_rm, tr_pc, va_rm, va_pc))

                            if do_print:
                                tqdm.write(f"[{k:4d}] TR rmse={tr_rm:.4f} pcc={tr_pc:.4f} | VA rmse={va_rm:.4f} pcc={va_pc:.4f}")

                            # checkpoint: latest (rolling)
                            save_ckpt(cp["latest"], k, self.theta, self.w, self.b, best_val)

                            # checkpoint: best
                            if obj < best_val:
                                best_val = obj
                                save_ckpt(cp["best"], k, self.theta, self.w, self.b, best_val)
                                # reset patience
                                patience = max(10, patience)  # ensure positive
                                patience_left = patience
                            else:
                                # patience accounting
                                if 'patience_left' not in locals():
                                    patience_left = patience
                                patience_left -= 1
                                if patience_left <= 0:
                                    tqdm.write("Early stopping (patience).")
                                    break
                        else:
                            if do_print:
                                tqdm.write(f"[{k:4d}] TR rmse={tr_rm:.4f} pcc={tr_pc:.4f}")

                # periodic checkpoint even without val (safety)
                if (k % checkpoint_every == 0) and (X_val_th is None):
                    save_ckpt(cp["latest"], k, self.theta, self.w, self.b, best_val)

        except KeyboardInterrupt:
            tqdm.write("Interrupted — saving latest checkpoint before exit.")
            save_ckpt(cp["latest"], k, self.theta, self.w, self.b, best_val)

        # load best if exists
        if os.path.exists(cp["best"]):
            _, thb, wb, bb, _ = load_ckpt(cp["best"])
            self.theta, self.w, self.b = thb, wb, bb

        return hist


# -------------------- pipeline & run --------------------
def run_vqc_multiobs(
    data_path="final_processed_als_data.csv",
    n_qubits=4, pls_components=4, topk=16,
    fmap_reps=2, ansatz_reps=2, basis="Z", use_pairs=True,
    spsa_steps=900, batch_size=160, patience=120,
    spsa_a=0.10, spsa_c=0.12, spsa_A=80, spsa_alpha=0.602, spsa_gamma=0.101,
    l2=0.001, seeds=(13, 37, 91), n_boot=5000,
    # checkpointing
    checkpoint_dir: str = "checkpoints_vqc",
    checkpoint_every: int = 50,
    resume: bool = True
):
    t0 = time.time()
    X, y = load_numeric_xy(data_path)
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=42)
    print(f"Split: train={X_tr.shape[0]} test={X_te.shape[0]}")

    # z-score target on TRAIN ONLY (and apply to val/test)
    y_mu, y_sd = zscore_fit(y_tr)
    yz_tr = zscore_apply(y_tr, y_mu, y_sd)
    yz_te = zscore_apply(y_te, y_mu, y_sd)

    # feature selection (train-only)
    idxK, _ = select_topk_by_rf(X_tr, y_tr, k=topk)

    # train/val split inside train for early stopping
    X_trA, X_val, y_trA, y_val = train_test_split(X_tr, y_tr, test_size=0.20, random_state=123)
    yz_trA = zscore_apply(y_trA, y_mu, y_sd)
    yz_val = zscore_apply(y_val, y_mu, y_sd)

    # PLS -> angles [0, π] (fit on TR only)
    imp = SimpleImputer(strategy="median").fit(X_trA.iloc[:, idxK])
    std = StandardScaler().fit(imp.transform(X_trA.iloc[:, idxK]))
    XtrK = std.transform(imp.transform(X_trA.iloc[:, idxK]))
    XvaK = std.transform(imp.transform(X_val.iloc[:, idxK]))
    XteK = std.transform(imp.transform(X_te.iloc[:, idxK]))

    assert pls_components == n_qubits, "pls_components must equal n_qubits"
    pls = PLSRegression(n_components=pls_components, scale=False).fit(XtrK, y_trA)
    TrP = pls.transform(XtrK); VaP = pls.transform(XvaK); TeP = pls.transform(XteK)

    ang = MinMaxScaler(feature_range=(0.0, np.pi)).fit(TrP)
    TrTH = ang.transform(TrP); VaTH = ang.transform(VaP); TeTH = ang.transform(TeP)

    # try multiple random starts; keep best val
    best = None
    est = AerEstimator() if AER_OK else Estimator()

    for sd in seeds:
        print(f"\n--- Training VQC (seed={sd}) ---")
        vqc = VQRegressorSPSA_MultiObs(
            n_qubits=n_qubits, fmap_reps=fmap_reps, ansatz_reps=ansatz_reps,
            basis=basis, use_pairs=use_pairs, estimator=est, seed=sd, l2=l2
        )
        vqc.fit(
            TrTH, yz_trA, X_val_th=VaTH, yz_val=yz_val,
            steps=spsa_steps, batch_size=batch_size,
            a=spsa_a, c=spsa_c, A=spsa_A, alpha=spsa_alpha, gamma=spsa_gamma,
            theta_init=None, seed=sd, verbose=True, patience=patience,
            checkpoint_dir=checkpoint_dir, checkpoint_every=checkpoint_every, resume=resume
        )

        # val performance (original units)
        yv_hat_z = vqc.predict_z(VaTH)
        yv_hat   = zscore_invert(yv_hat_z, y_mu, y_sd)
        va_rm, va_pc = rmse(y_val, yv_hat), safe_pcc(y_val, yv_hat)
        print(f"(seed={sd}) VAL  RMSE={va_rm:.4f}  PCC={va_pc:.4f}")

        item = (va_rm, -(va_pc), vqc, sd)   # sort by rmse asc, pcc desc
        if (best is None) or (item < best):
            best = item

    _, _, vqc_best, best_seed = best
    print(f"\n>>> Best seed: {best_seed}")

    # test once with best model (paper-style)
    yte_hat_z = vqc_best.predict_z(TeTH)
    yte_hat   = zscore_invert(yte_hat_z, y_mu, y_sd)
    test_rmse = rmse(y_te, yte_hat)
    test_pcc  = safe_pcc(y_te, yte_hat)
    ci_rm = bootstrap_ci(y_te, yte_hat, rmse,     n_boot=n_boot, seed=202)
    ci_pc = bootstrap_ci(y_te, yte_hat, safe_pcc, n_boot=n_boot, seed=303)

    print("\n===== VQC (Multi-Obs) TEST PERFORMANCE =====")
    print(f"RMSE={test_rmse:.4f}  PCC={test_pcc:.4f}")
    print(f"RMSE 95% CI [{ci_rm[0]:.4f}, {ci_rm[1]:.4f}]")
    print(f"PCC  95% CI [{ci_pc[0]:.4f}, {ci_pc[1]:.4f}]")

    D = vqc_best.D
    print("\nModel: n_qubits=%d  fmap_reps=%d  ansatz_reps=%d  basis=%s  pairs=%s  observables=%d"
          % (n_qubits, fmap_reps, ansatz_reps, basis, str(use_pairs), D))
    print("Time: %.1fs   AER=%s" % (time.time()-t0, str(AER_OK)))

    return dict(rmse=test_rmse, pcc=test_pcc, rmse_ci=ci_rm, pcc_ci=ci_pc, seed=best_seed)


if __name__ == "__main__":
    # ==== STRONGER, paper-chasing run (slower) ====
    # _ = run_vqc_multiobs(
    #     data_path="final_processed_als_data.csv",
    #     n_qubits=6, pls_components=6, topk=24,
    #     fmap_reps=3, ansatz_reps=3, basis="ZX", use_pairs=True,
    #     spsa_steps=1200, batch_size=192, patience=180,
    #     spsa_a=0.06, spsa_c=0.12, spsa_A=150, spsa_alpha=0.602, spsa_gamma=0.101,
    #     l2=0.003, seeds=(13, 37, 91), n_boot=5000,
    #     checkpoint_dir="checkpoints_vqc", checkpoint_every=50, resume=True
    # )

    # ==== Faster sanity run (fits in typical Colab session) ====
    _ = run_vqc_multiobs(
        data_path="final_processed_als_data.csv",
        n_qubits=4, pls_components=4, topk=16,
        fmap_reps=2, ansatz_reps=3, basis="ZX", use_pairs=True,
        spsa_steps=500, batch_size=192, patience=140,
        spsa_a=0.08, spsa_c=0.12, spsa_A=120, spsa_alpha=0.602, spsa_gamma=0.101,
        l2=0.002, seeds=(13, 37), n_boot=2000,
        checkpoint_dir="checkpoints_vqc", checkpoint_every=50, resume=True
    )

✓ Data: X_num=(1897, 343) (from 346 total), y=(1897,)
Split: train=1517 test=380
✓ Top-16 numeric features: ['alsfrs_ALSFRS_Total_slope', 'fvc_FVC_Liters_slope', 'fvc_FVC_Liters_std', 'alsfrs_ALSFRS_Total_std', 'vitals_Blood_Pressure_Systolic_slope', 'Age', 'vitals_Weight_slope', 'alsfrs_ALSFRS_Delta_std', 'labs_White Blood Cell (WBC)_first', 'labs_Phosphorus_median', 'vitals_Blood_Pressure_Diastolic_std', 'vitals_Respiratory_Rate_slope', 'vitals_Pulse_median', 'vitals_Blood_Pressure_Diastolic_slope', 'alsfrs_Q3_Swallowing_slope', 'alsfrs_ALSFRS_Total_median']

--- Training VQC (seed=13) ---
[   1] TR rmse=0.9877 pcc=0.1782 | VA rmse=1.0336 pcc=-0.0126
[  10] TR rmse=0.9874 pcc=0.1792 | VA rmse=1.0331 pcc=-0.0109
[  20] TR rmse=0.9873 pcc=0.1797 | VA rmse=1.0331 pcc=-0.0106
[  30] TR rmse=0.9871 pcc=0.1805 | VA rmse=1.0329 pcc=-0.0091
[  40] TR rmse=0.9868 pcc=0.1807 | VA rmse=1.0324 pcc=-0.0082
[  50] TR rmse=0.9867 pcc=0.1805 | VA rmse=1.0321 pcc=-0.0085
[  60] TR rmse=0.9864 pcc=0.1